In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings("ignore")

In [3]:
import featuretools as ft

In [2]:
train_df = pd.read_csv('input/train.csv', na_values=['NAN','NA','NaN','na','nan'])
test_df = pd.read_csv('input/test.csv', na_values=['NAN','NA','NaN','na','nan'])
X_all = pd.read_csv('input/X_all.csv', na_values=['NAN','NA','NaN','na','nan'])

In [4]:
es = ft.EntitySet(id="hospitals")

In [5]:
es = es.entity_from_dataframe(entity_id="clients",
                                        dataframe=X_all,
                                        index="inst_id",
                                        variable_types={"sido": ft.variable_types.Categorical,
                                                        "employee1": ft.variable_types.Categorical,
                                                        "employee2": ft.variable_types.Categorical,
                                                        "instkind": ft.variable_types.Categorical,
                                                        "ownerChange": ft.variable_types.Categorical})

In [6]:
es = es.normalize_entity(base_entity_id="clients",
                            new_entity_id="city",
                            index="sido")
es = es.normalize_entity(base_entity_id="clients",
                            new_entity_id="kind",
                            index="instkind")
es = es.normalize_entity(base_entity_id="clients",
                            new_entity_id="change",
                            index="ownerChange")

In [7]:
es

Entityset: hospitals
  Entities:
    change [Rows: 3, Columns: 1]
    kind [Rows: 8, Columns: 1]
    clients [Rows: 428, Columns: 57]
    city [Rows: 17, Columns: 1]
  Relationships:
    clients.sido -> city.sido
    clients.instkind -> kind.instkind
    clients.ownerChange -> change.ownerChange

In [8]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                        target_entity="clients",
                                        agg_primitives=["count", "mean", "sum", "std", "median", "mode"],
                                        trans_primitives=["month","diff","week"],
                                        max_depth=2)

In [9]:
print('Original',X_all.shape, 'Featured', feature_matrix.shape)

Original (428, 57) Featured (428, 822)


In [10]:
feature_matrix.head()

,noi1,tanAsset1,debt2,netAsset1,profit1,longLoan1,longLoan2,noi2,receivableL2,nonCAsset1,...,kind.STD(clients.profit2),city.STD(clients.longLoan1),change.MEDIAN(clients.revenue1),kind.STD(clients.salescost2),city.MEDIAN(clients.salescost1),kind.SUM(clients.sga1),city.SUM(clients.OnonCAsset2),kind.SUM(clients.quickAsset2),kind.SUM(clients.salescost1),change.MEDIAN(clients.shortLoan1)
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1.565244e+07,2.360684e+09,7.589937e+08,2.844460e+09,225169678.0,3.510000e+08,3.900000e+08,1.619468e+07,0.0,2.514586e+09,...,5.806077e+08,4.922569e+09,6.303899e+09,6.212511e+08,108424820.0,1.112561e+12,2.742782e+09,3.099628e+11,6.805439e+10,517485993.0
2,4.314093e+09,4.629931e+10,5.540643e+10,6.295769e+09,503669196.0,6.319097e+09,4.690000e+09,4.076448e+09,0.0,4.807282e+10,...,2.450469e+09,8.311996e+09,6.303899e+09,1.803727e+10,154633402.0,1.885607e+12,1.634169e+10,5.315738e+11,5.078804e+11,517485993.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.450469e+09,3.068164e+09,6.303899e+09,1.803727e+10,367242573.0,1.885607e+12,5.939987e+09,5.315738e+11,5.078804e+11,517485993.0
4,7.615600e+04,1.204810e+08,0.000000e+00,3.005088e+08,41864754.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.204810e+08,...,5.806077e+08,7.233080e+09,6.303899e+09,6.212511e+08,277802982.0,1.112561e+12,1.958362e+10,3.099628e+11,6.805439e+10,517485993.0
5,6.871024e+07,6.678648e+10,6.730838e+10,1.470447e+10,452555746.0,1.720000e+10,1.830000e+10,1.918022e+08,0.0,7.511010e+10,...,1.484833e+09,1.003284e+10,6.303899e+09,2.863205e+09,0.0,1.372868e+12,7.157730e+09,3.799238e+11,2.075124e+11,517485993.0


In [11]:
feature_defs

[<Feature: noi1>,
 <Feature: tanAsset1>,
 <Feature: debt2>,
 <Feature: netAsset1>,
 <Feature: profit1>,
 <Feature: longLoan1>,
 <Feature: longLoan2>,
 <Feature: noi2>,
 <Feature: receivableL2>,
 <Feature: nonCAsset1>,
 <Feature: liquidLiabilities1>,
 <Feature: liquidAsset1>,
 <Feature: inventoryAsset1>,
 <Feature: ownerChange>,
 <Feature: NCLiabilities2>,
 <Feature: liquidAsset2>,
 <Feature: salary2>,
 <Feature: noe2>,
 <Feature: quickAsset1>,
 <Feature: shortLoan1>,
 <Feature: interest1>,
 <Feature: employee1>,
 <Feature: revenue1>,
 <Feature: salescost1>,
 <Feature: shortLoan2>,
 <Feature: NCLiabilities1>,
 <Feature: ctax1>,
 <Feature: receivableL1>,
 <Feature: profit2>,
 <Feature: nonCAsset2>,
 <Feature: surplus2>,
 <Feature: instkind>,
 <Feature: salary1>,
 <Feature: revenue2>,
 <Feature: tanAsset2>,
 <Feature: netAsset2>,
 <Feature: sido>,
 <Feature: salescost2>,
 <Feature: bedCount>,
 <Feature: interest2>,
 <Feature: sga2>,
 <Feature: sga1>,
 <Feature: noe1>,
 <Feature: employee2

In [12]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)

In [13]:
print('Original',X_all.shape, 'Featured', feature_matrix.shape, 'Featured Categorical', feature_matrix_enc.shape)

Original (428, 57) Featured (428, 822) Featured Categorical (428, 930)


In [14]:
features_enc

[<Feature: noi1>,
 <Feature: tanAsset1>,
 <Feature: debt2>,
 <Feature: netAsset1>,
 <Feature: profit1>,
 <Feature: longLoan1>,
 <Feature: longLoan2>,
 <Feature: noi2>,
 <Feature: receivableL2>,
 <Feature: nonCAsset1>,
 <Feature: liquidLiabilities1>,
 <Feature: liquidAsset1>,
 <Feature: inventoryAsset1>,
 <Feature: ownerChange = same>,
 <Feature: ownerChange = change>,
 <Feature: ownerChange = unknown>,
 <Feature: NCLiabilities2>,
 <Feature: liquidAsset2>,
 <Feature: salary2>,
 <Feature: noe2>,
 <Feature: quickAsset1>,
 <Feature: shortLoan1>,
 <Feature: interest1>,
 <Feature: employee1 = 79.0>,
 <Feature: employee1 = 73.0>,
 <Feature: employee1 = 54.0>,
 <Feature: employee1 = 62.0>,
 <Feature: employee1 = 60.0>,
 <Feature: employee1 = 50.0>,
 <Feature: employee1 = 42.0>,
 <Feature: employee1 = 77.0>,
 <Feature: employee1 = 75.0>,
 <Feature: employee1 = 67.0>,
 <Feature: employee1 = unknown>,
 <Feature: revenue1>,
 <Feature: salescost1>,
 <Feature: shortLoan2>,
 <Feature: NCLiabilities1>

In [16]:
from category_encoders import *

In [17]:
cols_num = list(X_all.select_dtypes(include=['int64','float64']).columns)
cols_cat = list(X_all.select_dtypes(include=['object']).columns)

In [19]:
train_df['OC'][train_df['OC'] == 'open'] = 0
train_df['OC'][train_df['OC'] == ' close'] = 1

In [20]:
train_df['OC'].value_counts()

0    286
1     15
Name: OC, dtype: int64

In [24]:
X_train, X_test = feature_matrix_enc.iloc[:train_df.shape[0], :], feature_matrix_enc.iloc[train_df.shape[0]:, :] 

In [21]:
enc = TargetEncoder(cols=cols_cat).fit(train_df[cols_cat], train_df['OC'])

In [22]:
X_te = enc.transform(train_df[cols_cat])
X_test_te = enc.transform(test_df[cols_cat])

In [25]:
cols_cat_2 = list()
for c in cols_cat:
    cols_cat_2.append(c+'_enc')
cols = np.concatenate([X_train.columns.values,cols_cat_2],axis=0)

In [26]:
X = np.concatenate([X_train.as_matrix(), X_te.as_matrix()], axis=1)
X_test = np.concatenate([X_test.as_matrix(), X_test_te.as_matrix()], axis=1)

In [27]:
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)
X.columns = cols
X_test.columns = cols

In [28]:
X.head()

,noi1,tanAsset1,debt2,netAsset1,profit1,longLoan1,longLoan2,noi2,receivableL2,nonCAsset1,...,city.SUM(clients.OnonCAsset2),kind.SUM(clients.quickAsset2),kind.SUM(clients.salescost1),change.MEDIAN(clients.shortLoan1),sido_enc,openDate_enc,instkind_enc,employee1_enc,employee2_enc,ownerChange_enc
0,1.565244e+07,2.360684e+09,7.589937e+08,2.844460e+09,225169678.0,3.510000e+08,3.900000e+08,1.619468e+07,0.0,2.514586e+09,...,2.742782e+09,3.099628e+11,6.805439e+10,517485993.0,4.347826e-02,0.049834,0.034722,0.000896,0.013402,0.020243
1,4.314093e+09,4.629931e+10,5.540643e+10,6.295769e+09,503669196.0,6.319097e+09,4.690000e+09,4.076448e+09,0.0,4.807282e+10,...,1.634169e+10,5.315738e+11,5.078804e+11,517485993.0,8.852277e-17,0.049834,0.027027,0.049834,0.049834,0.020243
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.939987e+09,5.315738e+11,5.078804e+11,517485993.0,1.020408e-01,0.049834,0.034722,0.049834,0.000896,0.020243
3,7.615600e+04,1.204810e+08,0.000000e+00,3.005088e+08,41864754.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.204810e+08,...,1.958362e+10,3.099628e+11,6.805439e+10,517485993.0,6.149231e-06,0.049834,0.027027,0.049834,0.049834,0.020243
4,6.871024e+07,6.678648e+10,6.730838e+10,1.470447e+10,452555746.0,1.720000e+10,1.830000e+10,1.918022e+08,0.0,7.511010e+10,...,7.157730e+09,3.799238e+11,2.075124e+11,517485993.0,8.852277e-17,0.049834,0.027027,0.049834,0.049834,0.020243


In [29]:
X_test.head()

,noi1,tanAsset1,debt2,netAsset1,profit1,longLoan1,longLoan2,noi2,receivableL2,nonCAsset1,...,city.SUM(clients.OnonCAsset2),kind.SUM(clients.quickAsset2),kind.SUM(clients.salescost1),change.MEDIAN(clients.shortLoan1),sido_enc,openDate_enc,instkind_enc,employee1_enc,employee2_enc,ownerChange_enc
0,316745396.0,9.940461e+09,1.032818e+10,5.081292e+09,4.325110e+08,7.440000e+09,7.512550e+09,48705671.0,0.0,1.029148e+10,...,5.939987e+09,3.799238e+11,2.075124e+11,517485993.0,0.000006,0.049834,0.027027,0.049834,0.049834,0.020243
1,18142753.0,3.555309e+09,1.266739e+09,5.288964e+09,2.414326e+08,0.000000e+00,0.000000e+00,18171830.0,0.0,3.595463e+09,...,5.939987e+09,3.099628e+11,6.805439e+10,517485993.0,0.049834,0.049834,0.056818,0.049834,0.049834,0.020243
2,80103616.0,0.000000e+00,0.000000e+00,0.000000e+00,8.912300e+05,0.000000e+00,0.000000e+00,14024602.0,0.0,0.000000e+00,...,2.587327e+09,3.099628e+11,6.805439e+10,517485993.0,0.142857,0.049834,0.027027,0.049834,0.049834,0.049834
3,423628901.0,1.265639e+10,1.563217e+10,1.554179e+10,1.692204e+09,0.000000e+00,0.000000e+00,483447584.0,0.0,1.307623e+10,...,2.587327e+09,3.099628e+11,6.805439e+10,NaN,0.038462,0.049834,0.027027,0.049834,0.049834,0.020243
4,823222665.0,4.734251e+09,4.494217e+09,2.240287e+09,3.548233e+07,1.594400e+09,1.740600e+09,404832240.0,0.0,5.247103e+09,...,5.939987e+09,3.799238e+11,2.075124e+11,517485993.0,0.111111,0.049834,0.027027,0.049834,0.049834,0.020243


In [30]:
X.to_csv('input/X.csv', index=False)
X_test.to_csv('input/X_test.csv', index=False)